# neuroNumber 2 rotated
### Version 0.2
The two hidden layers model for a machine learning.

In training process each unit used in -10/0/10 deg. positions.

In [1]:
import numpy  # version 1.19.3 need to fix error on windows
import scipy.ndimage
from scipy.special import expit
import matplotlib.pyplot as plt
import time

In [5]:
class Neural:
    
    def __init__(self, 
                 input_nodes: int,
                 hidden_nodes_1: int, 
                 hidden_nodes_2: int,
                 output_nodes: int, 
                 learn_rate: float):
        
        self.input_nodes = input_nodes
        self.hidden_nodes_1 = hidden_nodes_1
        self.hidden_nodes_2 = hidden_nodes_2
        self.output_nodes = output_nodes
        self.lr = learn_rate
        
        self.wih = numpy.random.normal(
            0.0, 
            pow(self.hidden_nodes_1, -0.5),
            (self.hidden_nodes_1, self.input_nodes)
        )
        
        self.whh = numpy.random.normal(
            0.0, 
            pow(self.hidden_nodes_2, -0.5),
            (self.hidden_nodes_2, self.hidden_nodes_1)
        )
        
        self.who = numpy.random.normal(
            0.0, 
            pow(self.output_nodes, -0.5),
            (self.output_nodes, self.hidden_nodes_2)
        )
        
        self.activation_fn = lambda x: expit(x)
    
    
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        hidden_outputs_1, \
        hidden_outputs_2, \
        final_outputs = self.get_outputs(inputs)
        
        output_errors = targets - final_outputs
        hidden_errors_2 = numpy.dot(self.who.T, output_errors)
        hidden_errors_1 = numpy.dot(self.whh.T, hidden_errors_2)
        
        self.who += self.lr * numpy.dot(
            (output_errors * final_outputs * (1.0 - final_outputs)),
            numpy.transpose(hidden_outputs_2)
            )
        
        self.whh += self.lr * numpy.dot(
            (hidden_errors_2 * hidden_outputs_2 * (1.0 - hidden_outputs_2)),
            numpy.transpose(hidden_outputs_1)
            )
        
        self.wih += self.lr * numpy.dot(
            (hidden_errors_1 * hidden_outputs_1 * (1.0 - hidden_outputs_1)),
            numpy.transpose(inputs)
            )

    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        _, _, final_outputs = self.get_outputs(inputs)
        
        return final_outputs
    
    
    def get_outputs(self, inputs):
        hidden_inputs_1 = numpy.dot(self.wih, inputs)
        hidden_outputs_1 = self.activation_fn(hidden_inputs_1)
        
        hidden_inputs_2 = numpy.dot(self.whh, hidden_outputs_1)
        hidden_outputs_2 = self.activation_fn(hidden_inputs_2)
        
        final_inputs = numpy.dot(self.who, hidden_outputs_2)
        final_outputs = self.activation_fn(final_inputs)
        
        return hidden_outputs_1, hidden_outputs_2, final_outputs

In [ ]:
# Training and test data get from here:
# https://pjreddie.com/projects/mnist-in-csv/
# and pun in workdir
# 
# ../
# neuro_number.ipynb
# mnist_test.csv
# mnist_train.csv

In [10]:
# Layers
input_nodes = 784     # 28*28 - sample size
hidden_nodes_1 = 196  # 784/4
hidden_nodes_2 = 49   # 196/4
output_nodes = 10     # n-digits

# Training dataset (60_000)
training_file_name = "mnist_train.csv"
with open(training_file_name, 'r') as training_data_file:
    training_data_list = training_data_file.readlines()
data_length = len(training_data_list)
    
# Training settings
learning_rate = 0.012
epochs = 4
part_of_data_to_train = 1   # setup part of training dataset 0-1
max_count = int(data_length * part_of_data_to_train)

# Init model
n = Neural(input_nodes, 
           hidden_nodes_1,
           hidden_nodes_2,
           output_nodes, 
           learning_rate)

# Addition info to object
n.data_length = max_count
n.training_file_name = training_file_name
n.epochs = epochs

start_time = time.time()

# Intro
print("=== TRAINING ===")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length} + 2 rotation each (+/- 10 deg.)")
print(f"start: {time.strftime('%m/%d/%Y, %H:%M:%S', time.localtime())}")
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes 1 = {n.hidden_nodes_1}")
print(f"hidden nodes 2 = {n.hidden_nodes_2}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")

# Training
for e in range(epochs):
    print(f"\n--- Epoch {e + 1} of {epochs} ---")
    
    count = 0
    for record in training_data_list[:max_count]:
        all_values = record.split(',')
        
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        
        if count%5000 == 0:
            print(f"[{count + 1} of {max_count}] Training ...")
        
        inputs_orig = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        n.train(inputs_orig, targets)
        
        inputs_reshape = inputs_orig.reshape(28, 28)
        
        inputs_plus10 = scipy.ndimage.interpolation.rotate(
            inputs_reshape, 
            10, 
            cval=0.01, 
            reshape=False
        ).reshape(784)
        n.train(inputs_plus10, targets)
        
        inputs_minus10 = scipy.ndimage.interpolation.rotate(
            inputs_reshape, 
            -10, 
            cval=0.01, 
            reshape=False
        ).reshape(784)
        n.train(inputs_minus10, targets)
        
        count +=1

# Outro
end_time = time.time()
print(f"end time: {time.strftime('%m/%d/%Y, %H:%M:%S', time.localtime())}")
print(f"elapsed time: {end_time - start_time:.2f} sec.")
print("\n=== FINISH ===")


[]
=== TRAINING ===
training file = mnist_train.csv
training data length = 0 + 2 rotation each (+/- 10 deg.)
start: 12/06/2020, 14:47:32

--- Model ---
input nodes = 784
hidden nodes 1 = 196
hidden nodes 2 = 49
output nodes = 10
learning rate = 0.012

--- Epoch 1 of 4 ---

--- Epoch 2 of 4 ---

--- Epoch 3 of 4 ---

--- Epoch 4 of 4 ---
end time: 12/06/2020, 14:47:32
elapsed time: 0.01 sec.

=== FINISH ===


In [8]:
test_file_name = "mnist_test.csv"

with open(test_file_name, 'r') as testing_data_file:
    testing_data_list = testing_data_file.readlines()
    

data_length = len(testing_data_list)
max_count = 10000
correct = 0.50

count = 0
good = 0
start_time = time.time()

print("=== TEST ===")
print(f"test file: {test_file_name}")
print(f"test data: {max_count} units")
print(f"correct value must be > {correct * 100}%")
print(f"start: {time.strftime('%m/%d/%Y, %H:%M:%S', time.localtime())}")
      
print("\n--- Model ---")
print(f"input nodes = {n.input_nodes}")
print(f"hidden nodes 1 = {n.hidden_nodes_1}")
print(f"hidden nodes 2 = {n.hidden_nodes_2}")
print(f"output nodes = {n.output_nodes}")
print(f"learning rate = {n.lr}")
print(f"training file = {n.training_file_name}")
print(f"training data length = {n.data_length}")
print(f"training epochs = {n.epochs}")
        
for record in testing_data_list[:max_count]:
    all_values = record.split(',')
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    if count%1000 == 0:
        print(f"[{count + 1} of {max_count}] Testing ...")
    
    res = list(n.query(inputs))
    max_value = max(res)
    idx = res.index(max_value)
    number = int(all_values[0])
    if (float(max_value) > correct) and idx == number:
        good += 1
        
    count +=1

end_time = time.time()

# Outro
print("\n--- Report ---")
print(f"end time: {time.strftime('%m/%d/%Y, %H:%M:%S', time.localtime())}")
print(f"elapsed time: {end_time - start_time:.2f} sec.")
print(f"correct values is {good} from {max_count} ({good/max_count * 100:.3f}%)")
print("\n=== FINISH ===")

=== TEST ===
test file: mnist_test.csv
test data: 10000 units
correct value must be > 50.0%
start: 12/06/2020, 14:44:21

--- Model ---
input nodes = 784
hidden nodes 1 = 196
hidden nodes 2 = 49
output nodes = 10
learning rate = 0.012
training file = mnist_train.csv
training data length = 0
training epochs = 4

--- Report ---
end time: 12/06/2020, 14:44:21
elapsed time: 0.00 sec.
correct values is 0 from 10000 (0.000%)

=== FINISH ===


### Efficiency >96%